# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/v1/rerank`(cross encoder rerank model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)
## To run qwen2.5-0.5b-instruct model on the Ascend-Npu, you can execute the following command:
# server_process, port = launch_server_cmd(
#     "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --device npu --tp 2 --attention-backend torch_native"
# )

wait_for_server(f"http://localhost:{port}")

[2025-07-09 05:07:14] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=31207, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=705025560, constrained_json_whitespace_p

[2025-07-09 05:07:26] Attention backend not set. Use fa3 backend by default.
[2025-07-09 05:07:26] Init torch distributed begin.


[2025-07-09 05:07:26] Init torch distributed ends. mem usage=0.02 GB


[2025-07-09 05:07:29] Load weight begin. avail mem=77.30 GB


[2025-07-09 05:07:30] Using model weights format ['*.safetensors']
[2025-07-09 05:07:31] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

[2025-07-09 05:07:31] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=46.47 GB


[2025-07-09 05:07:32] INFO:     Started server process [2931144]
[2025-07-09 05:07:32] INFO:     Waiting for application startup.
[2025-07-09 05:07:32] INFO:     Application startup complete.
[2025-07-09 05:07:32] INFO:     Uvicorn running on http://0.0.0.0:31207 (Press CTRL+C to quit)


[2025-07-09 05:07:33] INFO:     127.0.0.1:47500 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 05:07:33] INFO:     127.0.0.1:47502 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 05:07:33] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:07:33.745122


[2025-07-09 05:07:34] INFO:     127.0.0.1:47514 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 05:07:34] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-07-09 05:07:38] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:07:38.402886


[2025-07-09 05:07:38] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 5.92, #queue-req: 0, timestamp: 2025-07-09T05:07:38.700859


[2025-07-09 05:07:39] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 122.23, #queue-req: 0, timestamp: 2025-07-09T05:07:39.028105


[2025-07-09 05:07:39] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 121.22, #queue-req: 0, timestamp: 2025-07-09T05:07:39.358096
[2025-07-09 05:07:39] INFO:     127.0.0.1:46662 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.
- `preferred_sampling_params`: The default sampling params specified via `--preferred-sampling-params`. `None` is returned in this example as we did not explicitly configure it in server args.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["preferred_sampling_params"] is None
assert response_json.keys() == {
    "model_path",
    "is_generation",
    "tokenizer_path",
    "preferred_sampling_params",
}

[2025-07-09 05:07:39] INFO:     127.0.0.1:46672 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-07-09 05:07:39] INFO:     127.0.0.1:46684 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-07-09 05:07:39] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:07:39.503942


[2025-07-09 05:07:40] INFO:     127.0.0.1:46696 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-07-09 05:07:40] INFO:     127.0.0.1:46710 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-07-09 05:07:40] Cache flushed successfully!
[2025-07-09 05:07:40] INFO:     127.0.0.1:46724 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-07-09 05:07:40] Start update_weights. Load format=auto
[2025-07-09 05:07:40] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-07-09 05:07:40] Using model weights format ['*.safetensors']
[2025-07-09 05:07:40] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.19it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  5.18it/s]

[2025-07-09 05:07:40] Update weights end.
[2025-07-09 05:07:40] Cache flushed successfully!
[2025-07-09 05:07:40] INFO:     127.0.0.1:46734 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-07-09 05:07:40] Start update_weights. Load format=auto
[2025-07-09 05:07:40] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-07-09 05:07:41] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-07-09 05:07:41] INFO:     127.0.0.1:46742 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

[2025-07-09 05:07:41] Child process unexpectedly failed with exitcode=9. pid=2932057


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 05:07:48] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=33764, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=1007757954, co

[2025-07-09 05:07:55] Downcasting torch.float32 to torch.float16.


[2025-07-09 05:07:55] Overlap scheduler is disabled for embedding models.
[2025-07-09 05:07:56] Downcasting torch.float32 to torch.float16.
[2025-07-09 05:07:56] Attention backend not set. Use fa3 backend by default.
[2025-07-09 05:07:56] Init torch distributed begin.


[2025-07-09 05:07:56] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 05:08:06] Load weight begin. avail mem=62.07 GB


...

[2025-07-09 05:08:27] Load weight end. type=Qwen2ForCausalLM, dtype=torch.float16, avail mem=57.54 GB, mem usage=4.53 GB.
[2025-07-09 05:08:27] KV Cache is allocated. #tokens: 20480, K size: 0.27 GB, V size: 0.27 GB
[2025-07-09 05:08:27] Memory pool end. avail mem=56.72 GB


[2025-07-09 05:08:30] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=56.62 GB


[2025-07-09 05:08:37] INFO:     Started server process [2934195]
[2025-07-09 05:08:37] INFO:     Waiting for application startup.
[2025-07-09 05:08:37] INFO:     Application startup complete.
[2025-07-09 05:08:37] INFO:     Uvicorn running on http://0.0.0.0:33764 (Press CTRL+C to quit)


[2025-07-09 05:08:37] INFO:     127.0.0.1:37448 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 05:08:38] INFO:     127.0.0.1:37462 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 05:08:38] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:08:38.236252


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-07-09 05:08:42] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:08:42.947670
[2025-07-09 05:08:42] INFO:     127.0.0.1:37474 - "POST /encode HTTP/1.1" 200 OK
[2025-07-09 05:08:42] The server is fired up and ready to roll!
[2025-07-09 05:08:43] INFO:     127.0.0.1:37476 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

[2025-07-09 05:08:43] Child process unexpectedly failed with exitcode=9. pid=2934539
[2025-07-09 05:08:43] Child process unexpectedly failed with exitcode=9. pid=2934472


## v1/rerank (cross encoder rerank model)
Rerank a list of documents given a query using a cross-encoder model. Note that this API is only available for cross encoder model like [BAAI/bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3) with `attention-backend` `triton` and `torch_native`.


In [14]:
reranker_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path BAAI/bge-reranker-v2-m3 \
    --host 0.0.0.0 --disable-radix-cache --chunked-prefill-size -1 --attention-backend triton --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 05:08:53] server_args=ServerArgs(model_path='BAAI/bge-reranker-v2-m3', tokenizer_path='BAAI/bge-reranker-v2-m3', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='BAAI/bge-reranker-v2-m3', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=34572, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=477043442, constrained_json_whitespace_pattern=None,

[2025-07-09 05:09:01] Downcasting torch.float32 to torch.float16.


[2025-07-09 05:09:02] Overlap scheduler is disabled for embedding models.
[2025-07-09 05:09:02] Downcasting torch.float32 to torch.float16.
[2025-07-09 05:09:02] Init torch distributed begin.


[2025-07-09 05:09:02] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 05:09:04] Load weight begin. avail mem=61.47 GB
[2025-07-09 05:09:04] Using model weights format ['*.safetensors']


...

[2025-07-09 05:09:05] KV Cache is allocated. #tokens: 20480, K size: 0.94 GB, V size: 0.94 GB
[2025-07-09 05:09:05] Memory pool end. avail mem=58.33 GB


[2025-07-09 05:09:06] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=8194, available_gpu_mem=58.24 GB


[2025-07-09 05:09:07] INFO:     Started server process [2936979]
[2025-07-09 05:09:07] INFO:     Waiting for application startup.
[2025-07-09 05:09:07] INFO:     Application startup complete.
[2025-07-09 05:09:07] INFO:     Uvicorn running on http://0.0.0.0:34572 (Press CTRL+C to quit)
[2025-07-09 05:09:07] INFO:     127.0.0.1:43846 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 05:09:08] INFO:     127.0.0.1:43848 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 05:09:08] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:09:08.663916


[2025-07-09 05:09:09] INFO:     127.0.0.1:43860 - "POST /encode HTTP/1.1" 200 OK
[2025-07-09 05:09:09] The server is fired up and ready to roll!


In [15]:
# compute rerank scores for query and documents

url = f"http://localhost:{port}/v1/rerank"
data = {
    "model": "BAAI/bge-reranker-v2-m3",
    "query": "what is panda?",
    "documents": [
        "hi",
        "The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.",
    ],
}

response = requests.post(url, json=data)
response_json = response.json()
for item in response_json:
    print_highlight(f"Score: {item['score']:.2f} - Document: '{item['document']}'")

[2025-07-09 05:09:12] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:09:12.697960
[2025-07-09 05:09:12] Prefill batch. #new-seq: 1, #new-token: 43, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:09:12.721025
[2025-07-09 05:09:12] INFO:     127.0.0.1:43872 - "POST /v1/rerank HTTP/1.1" 200 OK


In [16]:
terminate_process(reranker_process)

[2025-07-09 05:09:12] Child process unexpectedly failed with exitcode=9. pid=2937456
[2025-07-09 05:09:12] Child process unexpectedly failed with exitcode=9. pid=2937390


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [17]:
# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 05:09:22] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=39918, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed

[2025-07-09 05:09:29] Overlap scheduler is disabled for embedding models.
[2025-07-09 05:09:29] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 05:09:29] Init torch distributed begin.


[2025-07-09 05:09:30] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 05:09:31] Load weight begin. avail mem=76.30 GB


[2025-07-09 05:09:32] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:01,  1.01s/it]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.46it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:03<00:00,  1.26it/s]

[2025-07-09 05:09:36] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=64.46 GB, mem usage=11.85 GB.


[2025-07-09 05:09:37] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-07-09 05:09:37] Memory pool end. avail mem=61.64 GB


[2025-07-09 05:09:37] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=60.58 GB


[2025-07-09 05:09:38] INFO:     Started server process [2939240]
[2025-07-09 05:09:38] INFO:     Waiting for application startup.
[2025-07-09 05:09:38] INFO:     Application startup complete.
[2025-07-09 05:09:38] INFO:     Uvicorn running on http://0.0.0.0:39918 (Press CTRL+C to quit)
[2025-07-09 05:09:38] INFO:     127.0.0.1:44426 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 05:09:39] INFO:     127.0.0.1:44428 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 05:09:39] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:09:39.713861


In [18]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-07-09 05:10:44] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, #token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:10:44.704952
[2025-07-09 05:10:44] INFO:     127.0.0.1:44444 - "POST /encode HTTP/1.1" 200 OK
[2025-07-09 05:10:44] The server is fired up and ready to roll!


[2025-07-09 05:10:59] INFO:     127.0.0.1:33546 - "POST /classify HTTP/1.1" 200 OK


In [19]:
terminate_process(reward_process)

[2025-07-09 05:10:59] Child process unexpectedly failed with exitcode=9. pid=2940187
[2025-07-09 05:10:59] Child process unexpectedly failed with exitcode=9. pid=2940028


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [20]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-07-09 05:11:06] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, skip_server_warmup=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, hybrid_kvcache_ratio=None, impl='auto', host='0.0.0.0', port=35114, nccl_port=None, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=366660617, constrained_json_whitespace_pattern=None,

[2025-07-09 05:11:13] Attention backend not set. Use flashinfer backend by default.
[2025-07-09 05:11:13] Init torch distributed begin.


[2025-07-09 05:11:14] Init torch distributed ends. mem usage=0.00 GB


[2025-07-09 05:11:16] Load weight begin. avail mem=78.48 GB


[2025-07-09 05:11:19] The weight of LmHead is not packed
[2025-07-09 05:11:19] Using model weights format ['*.safetensors']


...

[2025-07-09 05:11:27] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=47.28 GB


[2025-07-09 05:11:28] INFO:     Started server process [2947454]
[2025-07-09 05:11:28] INFO:     Waiting for application startup.
[2025-07-09 05:11:28] INFO:     Application startup complete.
[2025-07-09 05:11:28] INFO:     Uvicorn running on http://0.0.0.0:35114 (Press CTRL+C to quit)


[2025-07-09 05:11:28] INFO:     127.0.0.1:46410 - "GET /v1/models HTTP/1.1" 200 OK


[2025-07-09 05:11:29] INFO:     127.0.0.1:46426 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-07-09 05:11:29] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, #token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, timestamp: 2025-07-09T05:11:29.017384


[2025-07-09 05:11:30] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1c-gpu-0/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/triton_3_3_1/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


In [21]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

[2025-07-09 05:11:44] Resetting ExpertDistributionRecorder...
[2025-07-09 05:11:44] INFO:     127.0.0.1:46442 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-07-09 05:11:44] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, #token: 8, token usage: 0.00, #running-req: 1, #queue-req: 0, timestamp: 2025-07-09T05:11:44.874571


[2025-07-09 05:11:45] INFO:     127.0.0.1:46438 - "POST /generate HTTP/1.1" 200 OK
[2025-07-09 05:11:45] The server is fired up and ready to roll!


[2025-07-09 05:11:46] Decode batch. #running-req: 1, #token: 46, token usage: 0.00, cuda graph: False, gen throughput (token/s): 2.44, #queue-req: 0, timestamp: 2025-07-09T05:11:46.208137


[2025-07-09 05:11:47] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, cuda graph: False, gen throughput (token/s): 37.07, #queue-req: 0, timestamp: 2025-07-09T05:11:47.287076


[2025-07-09 05:11:48] Decode batch. #running-req: 1, #token: 126, token usage: 0.01, cuda graph: False, gen throughput (token/s): 35.24, #queue-req: 0, timestamp: 2025-07-09T05:11:48.422301


[2025-07-09 05:11:48] INFO:     127.0.0.1:39356 - "POST /generate HTTP/1.1" 200 OK


[2025-07-09 05:11:48] INFO:     127.0.0.1:39372 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-07-09 05:11:49] Write expert distribution to /tmp/expert_distribution_recorder_1752037909.6812549.pt
[2025-07-09 05:11:49] Resetting ExpertDistributionRecorder...
[2025-07-09 05:11:49] INFO:     127.0.0.1:39382 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [22]:
terminate_process(expert_record_server_process)

[2025-07-09 05:11:49] Child process unexpectedly failed with exitcode=9. pid=2947672
[2025-07-09 05:11:49] Child process unexpectedly failed with exitcode=9. pid=2947606
